# Create boxdata
* Input: label data
* Output: 3D npy file with CT iamge and label (pancreas, lesion)

In [1]:
import time
import os, glob, ntpath
import logging
from tqdm import tqdm
import numpy as np
import pandas as pd
import pydicom as dicom
import nrrd
import matplotlib.pyplot as plt
from datetime import datetime as ddt

In [2]:
border = np.array([0, 0, 0])
# box_save_path = '/home/d/pancreas/new_box_data/'
box_save_path = '/home/d/pancreas/box_data_test/'
# log_path = '/home/g/pancreas/proj/Alpha/log/'
log_path = '/home/u/sonic81518/project/pancreas/log/'

In [3]:
# Helpful functions
sortkey = lambda k: ntpath.basename(k).replace('.dcm', '').split('-')[-1]
sortkey_PC = lambda k: int(ntpath.basename(k).replace('.dcm', '').split('I')[-1])
get_slicelocation = lambda k: dicom.read_file(k, force=True)[0x0020, 0x1041].value
get_imageposition = lambda k: np.array(list(map(float, list(dicom.read_file(k, force=True)[0x0020, 0x0032].value))))
get_pixelspacing = lambda k: list(map(float, list(dicom.read_file(k, force=True)[0x0028, 0x0030].value)))

<h1> Logging System </h1>

In [4]:
log_name = ['boxdata', '%Y%m%d%H%M%S.log']
log_filename = ddt.now().strftime(log_path + '_'.join(log_name))

logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
    format='%(levelname)-8s: %(asctime)-12s: %(message)s'
)

In [5]:
logging.info("Create box data.")

<h1> Box Data </h1>

In [6]:
# Usage: Transfer dicom image's data to Hounsfield units (HU)
def get_pixels_hu(scans):
    # Convert to int16, should be possible as values should always be low enough (<32k)
    image = np.stack([s.pixel_array for s in scans]).astype(np.int16)
    
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(scans)):
        intercept = scans[slice_number].RescaleIntercept
        slope = scans[slice_number].RescaleSlope
        if slope != 1:
            image[slice_number] = (slope * image[slice_number].astype(np.float64)).astype(np.int16)
        image[slice_number] += np.int16(intercept)
    return np.array(image, dtype=np.int16)

In [7]:
def create_box_data(tumorpath, border, sortkey = sortkey):
    '''
    Usage: Create box data from tumorpath and add border
    '''
    
    logging.info("Start operating " + tumorpath)
    
    tumor_id = ntpath.basename(os.path.normpath(tumorpath))
    
    # Read label nrrd
    tumor_label, tumor_options = nrrd.read(tumorpath+'label.nrrd')
    label_shape = np.array(tumor_label.shape[1:]) if len(tumor_label.shape) == 4 else np.array(tumor_label.shape)
    
    # Get ordered path and find path order
    dcmpathes = sorted(glob.glob(tumorpath+'scans/*.dcm'), key=sortkey)
#     order = 'downup' if get_slicelocation(dcmpathes[0]) < get_slicelocation(dcmpathes[1]) else 'updown'
    order = 'downup' if get_imageposition(dcmpathes[0])[2] < get_imageposition(dcmpathes[1])[2] else 'updown'
    logging.info("{:<21}: {}".format("DICOM save order", order))
    
    if order == 'updown': dcmpathes = dcmpathes[::-1]
    assert get_imageposition(dcmpathes[0])[2] < get_imageposition(dcmpathes[1])[2], "Wrong order!"
    
    # Find each space relative origin and spacing
    img_origin = get_imageposition(dcmpathes[0])
    thickness = abs(get_imageposition(dcmpathes[1])[2] - get_imageposition(dcmpathes[0]))[2]
    img_spacing = np.array(get_pixelspacing(dcmpathes[0]) + [float(thickness)])
    
    seg_origin = np.array(tumor_options['space origin'], dtype=float)
    if tumor_options['space directions'][0] == 'none':
        seg_spacing = np.diag(np.array(tumor_options['space directions'][1:]).astype(float))
    else:
        seg_spacing = np.diag(np.array(tumor_options['space directions']).astype(float))
    
    # Calculate segmetation origin index in image voxel coordinate
    seg_origin_idx = np.round((seg_origin / seg_spacing - img_origin / img_spacing)).astype(int)
    
    logging.info("{:<21}: [{t[0]:<4.3f}, {t[1]:<4.3f}, {t[2]:<4.3f}]".format("Image origin", t=img_origin))
    logging.info("{:<21}: [{t[0]:<3d}, {t[1]:<3d}, {t[2]:<3d}]".format("Label shape", t=label_shape))
    logging.info("{:<21}: [{t[0]:<4.3f}, {t[1]:<4.3f}, {t[2]:<4.3f}]".format("Image spacing", t=img_spacing))
    logging.info("{:<21}: [{t[0]:<4.3f}, {t[1]:<4.3f}, {t[2]:<4.3f}]".format("Segment origin", t=seg_origin))
    logging.info("{:<21}: [{t[0]:<4.3f}, {t[1]:<4.3f}, {t[2]:<4.3f}]".format("Segment spacing", t=seg_spacing))
    logging.info("{:<21}: [{t[0]:<3d}, {t[1]:<3d}, {t[2]:<3d}]".format("Segment origin idx", t=seg_origin_idx))

    
    # Get box origin and length
    box_origin_idx = seg_origin_idx - border
    box_length = label_shape + 2 * border
    
    x_orgidx, y_orgidx, z_orgidx = box_origin_idx
    x_len, y_len, z_len = box_length
    x_border, y_border, z_border = border
    
    logging.info("{:<21}: [{t[0]:<3d}, {t[1]:<3d}, {t[2]:<3d}]".format("Box origin idx", t=box_origin_idx))
    logging.info("{:<21}: [{t[0]:<3d}, {t[1]:<3d}, {t[2]:<3d}]".format("Box length", t=box_length))

    
    base_tumor_path = box_save_path + tumor_id + '/'
    if not os.path.exists(base_tumor_path):
        os.mkdir(base_tumor_path)
        
    # Get DICOM scans and transfer to HU
    patient_scan = [dicom.read_file(dcmpath, force = True) for dcmpath in dcmpathes[z_orgidx: z_orgidx+z_len]]
    for scan in patient_scan:
        scan.file_meta.TransferSyntaxUID = dicom.uid.ImplicitVRLittleEndian
    patient_hu = get_pixels_hu(patient_scan)[:, y_orgidx: y_orgidx+y_len, x_orgidx: x_orgidx+x_len]
    patient_hu = patient_hu.transpose(2, 1, 0)
    np.save(base_tumor_path+'ctscan.npy', patient_hu)
    logging.info("Save CT scan numpy array.")
    
    category_cnt = tumor_label.shape[0] if tumor_options['dimension'] == 4 else 1
    category_names = [tumor_options['keyvaluepairs']['Segment{}_Name'.format(c)] for c in range(category_cnt)]
    
    logging.info("Segment category amount: {}".format(category_cnt))
    logging.info("Segment category names: {}".format(', '.join(category_names)))
    
    # Get pancreas label 
    for i, category_name in enumerate(category_names):
        category_label = np.zeros(box_length)
#         CHANGE HERE!!
        category_label[x_border: x_len-x_border, y_border: y_len-y_border, z_border: z_len-z_border] = tumor_label[i] if len(tumor_label.shape) == 4 else tumor_label
        np.save(base_tumor_path+'{}.npy'.format(category_name.replace('-', '').replace('_','').replace(' ','').replace('1','').lower()), category_label)
        logging.info("Save category {} numpy array.".format(category_name))

In [10]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/new_label_data/*/PT*/*nrrd')):
    create_box_data(tumorpath.replace('/label.nrrd','/'), border)
    cnt += 1
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

100%|██████████| 44/44 [00:53<00:00,  1.22s/it]

Done create 44 box data in 53.498143434524536 seconds


In [11]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/new_label_data/*/NP*/*nrrd')):
    create_box_data(tumorpath.replace('/label.nrrd','/'), border)
    cnt += 1
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

100%|██████████| 10/10 [00:09<00:00,  1.07it/s]

Done create 10 box data in 9.39674711227417 seconds


In [9]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/new_label_data/*/PC*/*nrrd')):
    create_box_data(tumorpath.replace('/label.nrrd','/'), border, sortkey = sortkey_PC)
    cnt += 1
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

100%|██████████| 22/22 [00:06<00:00,  3.51it/s]

Done create 22 box data in 6.299109935760498 seconds


In [8]:
st_tol = time.time()
cnt = 0
for tumorpath in tqdm(glob.glob('/home/d/pancreas/new_label_data/*/BT*/*nrrd')):
    create_box_data(tumorpath.replace('/label.nrrd','/'), border, sortkey = sortkey_PC)
    cnt += 1
print('Done create {} box data in {} seconds'.format(cnt, time.time()-st_tol))

100%|██████████| 6/6 [00:02<00:00,  2.41it/s]

Done create 6 box data in 2.549407958984375 seconds
